In [1]:
import ctf,random
import numpy as np
import numpy.linalg as la
from ctf import random as crandom
glob_comm = ctf.comm()

In [2]:
class UnitTests:
        
    def test_general_purturb1(self):
        n = random.randint(2,5) # arbitrary dimension
        r = 1
        sparsity = .2
        regParam = 2
    
        # arbitrary tensor
        T,dimensions = init_tensor(n)
        factors = init_factors(dimensions,r)
    
        import string
        S = string.ascii_lowercase
        S = S[:len(dimensions)]
        #T.i("ijk") << U.i("iu")*V.i("ju")*W.i("ku")
        tmp = 1
        for idx in range(len(factors)):
            tmp = tmp*factors[idx].i(S[idx]+'u')   
        T.i(S) << tmp 

        omega = updateOmega(dimensions,sparsity)
        
        # purturb the first factor matrix
        factors[0] += crandom.random((dimensions[0],r))*.01
        others = factors.copy()
        others.pop(0)
        other_dimensions = dimensions.copy()
        other_dimensions.pop(0)
        nU = updateFactor(T,factors[0],others,regParam,omega,other_dimensions,r,S,0)
        
        nT = ctf.tensor(tuple(dimensions))
        tmp = nU.i('au')
        for idx in range(1,len(factors)):
            tmp = tmp*factors[idx].i(S[idx]+'u')   
        nT.i(S) << tmp 
        
        assert(ctf.all(ctf.abs(nT - T < 1e-10)))
        print("passed test: test_general_purturb1")
            
    
    def test_general_purturbn(self):
        n = random.randint(2,5) # arbitrary dimension
        r = 1
        sparsity = .2
        regParam = 2
    
        # arbitrary tensor
        T,dimensions = init_tensor(n)
        factors = init_factors(dimensions,r)
    
        import string
        S = string.ascii_lowercase
        S = S[:len(dimensions)]
        #T.i("ijk") << U.i("iu")*V.i("ju")*W.i("ku")
        tmp = 1
        for idx in range(len(factors)):
            tmp = tmp*factors[idx].i(S[idx]+'u')   
        T.i(S) << tmp 

        omega = updateOmega(dimensions,sparsity)
        
        # purturb the all n factor matrixces
        for i in range(len(factors)):
            factors[i] += crandom.random((dimensions[i],r))*.01
        
        others = factors.copy()
        others.pop(0)
        other_dimensions = dimensions.copy()
        other_dimensions.pop(0)
        nU = updateFactor(T,factors[0],others,regParam,omega,other_dimensions,r,S,0)
        
        nT = ctf.tensor(tuple(dimensions))
        tmp = nU.i('au')
        for idx in range(1,len(factors)):
            tmp = tmp*factors[idx].i(S[idx]+'u')   
        nT.i(S) << tmp 
        
        assert(ctf.all(ctf.abs(nT - T < 1e-10)))
        print("passed test: test_general_purturbn")
            
        
    def runAllTests(self):
        self.test_general_purturb1()
        self.test_general_purturbn()

In [3]:
def updateFactor(T,curr,others,regParam,omega,other_dimensions,r,s,idx):
    '''Update single factor matrix by using the formula'''
    
    other_dimensions_ = other_dimensions.copy()
    other_dimensions_.append(r)
    M = ctf.tensor((tuple(other_dimensions_)))
    
    # M.i("jku") << V.i("ju")*W.i("ku")
    S = s[:idx] + s[idx + 1:]
    temp = 1
    for k in range(len(others)):
        temp = temp*others[k].i(S[k]+'u')
    M.i(S+'u') << temp
    
    D = np.prod(other_dimensions)
    [U_,S_,V_] = ctf.svd(M.reshape((D,r)))
    S_ = S_/(S_*S_ + regParam*ctf.ones(r))
    curr.set_zero()
    curr.i(s[idx]+'u') << V_.i("vu")*S_.i("v")*U_.reshape((tuple(other_dimensions_))).i(S+'v')*T.i(s)
    #U.i("iu") << V_.i("vu")*S_.i("v")*U_.reshape((J,K,r)).i("jkv")*T.i("ijk")
    #curr *= normalize(curr,r)
    return curr

In [4]:
def getALSCtf(T,factors,regParam,omega,dimensions,r):
    '''
    Tensor completion using ALS
    '''
    it = 0
    E = ctf.tensor(tuple(dimensions))
    curr_err_norm = 0
    
    import string
    s = string.ascii_lowercase
    s = s[:len(dimensions)]
    
    e = 1
    for idx in range(len(factors)):
        #print(s[idx]+'u')
        e = e*factors[idx].i(s[idx]+'u')
        curr_err_norm += ctf.vecnorm(factors[idx])
    E.i(s) << T.i(s) - e
    #E.i("ijk") << T.i("ijk") - U.i("iu")*V.i("ju")*W.i("ku")
    curr_err_norm = ctf.vecnorm(E) + curr_err_norm*regParam
    
    while True:
        for idx in range(len(factors)):
            others = factors.copy()
            others.pop(idx)
            other_dimensions = dimensions.copy()
            other_dimensions.pop(idx)
        
            factors[idx] = updateFactor(T,factors[idx],others,regParam,omega,other_dimensions,r,s,idx)

        E.set_zero()
        next_err_norm = 0
        ne = 1
        for idx in range(len(factors)):
            ne = ne*factors[idx].i(s[idx]+'u')
            next_err_norm += ctf.vecnorm(factors[idx])
        E.i(s) << (T.i(s) - ne)
        next_err_norm = ctf.vecnorm(E) + next_err_norm*regParam
  
        if abs(curr_err_norm - next_err_norm) < .001 or it > 10:
            break
         
        print(curr_err_norm, next_err_norm)
        curr_err_norm = next_err_norm
        it += 1
    
    print("Number of iterations: ", it)
    return factors

In [5]:
def normalize(Z,r):
    norms = ctf.tensor(r)
    norms.i("u") << Z.i("pu")*Z.i("pu")
    norms = 1./norms**.5
    X = ctf.tensor(copy=Z)
    Z.set_zero()
    Z.i("pu") << X.i("pu")*norms.i("u")
    return 1./norms

In [6]:
def init_tensor(n):
    dimensions = []
    while n>0:
        r = random.randint(1,10)
        dimensions.append(r)
        n -= 1
    T = ctf.tensor(tuple(dimensions))
    #T.fill_random(0,1)
    return T,dimensions


def init_factors(dimensions,r):
    D = dimensions.copy()
    ctf.random.seed(42)
    factors = []
    while len(D) != 0:
        I = D.pop(0)
        U = ctf.random.random((I,r))
        #normalize(U)
        factors.append(U)
    return factors


def updateOmega(dimensions,sparsity):
    '''
    Gets a random subset of rows
    '''
    Actf = ctf.tensor(tuple(dimensions),sp=True)
    Actf.fill_sp_random(1,1,sparsity)
    #omegactf = ((Actf > 0)*ctf.astensor(1.))
    return Actf

In [11]:
def main():
    
    ut = UnitTests()
    ut.runAllTests()
    
    n = random.randint(2,5) # arbitrary dimension
    #n = 3
    r = 1
    sparsity = .2
    regParam = 20
    
    # arbitrary tensor
    T,dimensions = init_tensor(n)
    print("dimensions: ", tuple(dimensions))
    factors = init_factors(dimensions,r)
            
    import string
    s_tmp = string.ascii_lowercase
    s_tmp = s_tmp[:len(dimensions)]
    #T.i("ijk") << U.i("iu")*V.i("ju")*W.i("ku")
    tmp = 1
    for idx in range(len(factors)):
        tmp = tmp*factors[idx].i(s_tmp[idx]+'u')   
    T.i(s_tmp) << tmp 
    
    omega = updateOmega(dimensions,sparsity)
        
    getALSCtf(T,factors,regParam,omega,dimensions,r)
    
    

In [12]:
main()

passed test: test_general_purturb1
passed test: test_general_purturbn
dimensions:  (8, 4, 7, 10, 3)
127.76543917270438 6.363463413635435
6.363463413635435 2.4242396042258223
Number of iterations:  2
